In [1]:
%cd ..

/home/eli/AnacondaProjects/categorical_bpl


In [2]:
import argparse
import collections
import pyro
import torch
import numpy as np
import data_loader.data_loaders as module_data
import model.model as module_arch
from parse_config import ConfigParser
from trainer import Trainer

In [3]:
%matplotlib inline

In [4]:
# pyro.enable_validation(True)
# torch.autograd.set_detect_anomaly(True)

In [5]:
# fix random seeds for reproducibility
SEED = 123
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(SEED)

In [6]:
Args = collections.namedtuple('Args', 'config resume device')
config = ConfigParser.from_args(Args(config='chemical_config.json', resume=None, device=None))

In [7]:
logger = config.get_logger('train')

In [8]:
# setup data_loader instances
data_loader = config.init_obj('data_loader', module_data)
valid_data_loader = data_loader.split_validation()

In [9]:
# build model architecture, then print to console
model = config.init_obj('arch', module_arch, len(data_loader.dataset.alphabet), data_loader.dataset.max_len)

In [10]:
optimizer = pyro.optim.ReduceLROnPlateau({
    'optimizer': torch.optim.Adam,
    'optim_args': {
        "lr": 1e-3,
        "weight_decay": 0,
        "amsgrad": True
    },
    "patience": 500,
    "factor": 0.1,
    "verbose": True,
})

In [11]:
# optimizer = config.init_obj('optimizer', pyro.optim)

In [12]:
trainer = Trainer(model, [], optimizer, config=config,
                  data_loader=data_loader,
                  valid_data_loader=valid_data_loader,
                  lr_scheduler=optimizer, log_images=False, log_step=128)

In [13]:
trainer.train()

Train Epoch: 1 [256/118836 (0%)] Loss: 15535.135742
Train Epoch: 1 [33024/118836 (28%)] Loss: 13563.041992
Train Epoch: 1 [65792/118836 (55%)] Loss: 13118.084961
Train Epoch: 1 [98560/118836 (83%)] Loss: 12886.484375
    epoch          : 1
    loss           : 13418.745990681864
    val_loss       : 12718.411792219049
    val_log_likelihood: -12618.686537653795
    val_log_marginal: -12650.827778787123
Train Epoch: 2 [256/118836 (0%)] Loss: 12673.580078
Train Epoch: 2 [33024/118836 (28%)] Loss: 12660.777344
Train Epoch: 2 [65792/118836 (55%)] Loss: 12710.605469
Train Epoch: 2 [98560/118836 (83%)] Loss: 12652.155273
    epoch          : 2
    loss           : 12646.914833572424
    val_loss       : 12624.462029577533
    val_log_likelihood: -12579.842377804489
    val_log_marginal: -12604.848000515178
Train Epoch: 3 [256/118836 (0%)] Loss: 12658.320312
Train Epoch: 3 [33024/118836 (28%)] Loss: 12622.830078
Train Epoch: 3 [65792/118836 (55%)] Loss: 12603.634766
Train Epoch: 3 [98560/1188

In [14]:
model.cpu()

SelfiesAutoencodingModel(
  (_operad): FreeOperad(
    (generator_0): RecurrentDecoder(
      (recurrence): GRU(32, 64, batch_first=True)
      (decoder): Sequential(
        (0): Linear(in_features=64, out_features=18, bias=True)
        (1): Softmax(dim=-1)
      )
    )
    (generator_1): RecurrentDecoder(
      (recurrence): GRU(32, 64, batch_first=True)
      (decoder): Sequential(
        (0): Linear(in_features=64, out_features=18, bias=True)
        (1): Softmax(dim=-1)
      )
    )
    (generator_2): RecurrentDecoder(
      (recurrence): GRU(32, 64, num_layers=2, batch_first=True)
      (decoder): Sequential(
        (0): Linear(in_features=128, out_features=18, bias=True)
        (1): Softmax(dim=-1)
      )
    )
    (generator_3): RecurrentDecoder(
      (recurrence): GRU(32, 64, num_layers=2, batch_first=True)
      (decoder): Sequential(
        (0): Linear(in_features=128, out_features=18, bias=True)
        (1): Softmax(dim=-1)
      )
    )
    (generator_4): Recurren